## Get the pvalue and zscore files of genes for traits


In [1]:
R.home()
.Library

[1] "/Users/hopekirby/miniconda3/envs/r-kernel/lib/R"

[1] "/Users/hopekirby/miniconda3/envs/r-kernel/lib/R/library"

In [2]:
library(data.table)
library(tidyverse)
library(Matrix)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()

Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




## 1. Get MAGMA based Gene Symbols

In [65]:
# READ IN GENE SCORES FROM MAGMA
dir <- "/Users/hopekirby/Desktop/SCRNA-GWAS-Benchmarking/output/MAGMA/RA/"
genes_out = fread(paste0(dir,"10kb_win.genes.out"))
genes_out[1:2,]

GENE,CHR,START,STOP,NSNPS,NPARAM,N,ZSTAT,P
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
148398,1,849993,889961,100,14,97173,-1.15290,0.87552
26155,1,869583,904679,88,9,97173,-0.15151,0.56021


In [4]:
dir_mag <- "/Users/hopekirby/Desktop/SC_GWAS_Bench/data/MAGMA/NCBI38/"
genes_loc = fread(paste0(dir_mag,"NCBI38.gene.loc"))
dim(genes_loc)
colnames(genes_loc) <- c('ENTREZ_ID', 'Chr', 'Start', 'Stop', 'Strand', 'SYMBOL')

hg38_list <- genes_loc$SYMBOL
length(hg38_list)

dir_mag <- "/Users/hopekirby/Desktop/SC_GWAS_Bench/data/MAGMA/NCBI37.3/"
genes_loc = fread(paste0(dir_mag,"NCBI37.3.gene.loc"))
dim(genes_loc)
colnames(genes_loc) <- c('ENTREZ_ID', 'Chr', 'Start', 'Stop', 'Strand', 'SYMBOL')

hg37_list <- genes_loc$SYMBOL
length(hg37_list)

[1] 20137     6

[1] 20137

[1] 19427     6

[1] 19427

In [66]:
# make a dictionary of entrez_id to Symbol
L <- c(genes_loc$SYMBOL)
names(L) <- genes_loc$ENTREZ_ID
L[1:4]

# Add SYmbol Column
genes_out <- genes_out %>% 
  mutate(
    SYMBOL = recode(GENE, !!!L)
  )
genes_out[1:3,]

79501      100996442         729759          81399 
       "OR4F5" "LOC100996442"       "OR4F29"       "OR4F16"

GENE,CHR,START,STOP,NSNPS,NPARAM,N,ZSTAT,P,SYMBOL
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>
148398,1,849993,889961,100,14,97173,-1.152900,0.87552,SAMD11
26155,1,869583,904679,88,9,97173,-0.151510,0.56021,NOC2L
339451,1,885967,911099,77,10,97173,-0.025141,0.51003,KLHL17


In [67]:
# Order by p-value 
genes_out <- genes_out[order(genes_out$P),]
top_1000 <- genes_out$SYMBOL[1:1000]
gene_list_sumstats <- genes_out$SYMBOL
length(gene_list_sumstats)
length(unique(gene_list_sumstats))

[1] 18065

[1] 18065

## 2. Fix gene aliases
1. Fix Aliases from NCBI synonyms file (ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz) 
2. Fix remaining protein-encoding genes by looking up aliases on Genecards
3. Remove duplicates
4. Only include genes that are also found in the scRNA-seq dataset

In [7]:
# get the list of genes in the SC data
dir_data <- "/Users/hopekirby/Desktop/ZhangS23/data/amp2/"
exp_norm <- readRDS(paste0(dir_data, "qc_mRNA_314011cells_log_normalized_matrix_2023-03-15.rds"))
# dir_data <- "../../data/SC_data/Simile_UC/"
# exp_norm <- readRDS(paste0(dir_data, "exprs_norm_qc_ulcerative_colitis_gut.rds"))
exp_norm[1:2,1:2]
gene_list_RA <- rownames(exp_norm)
gene_list_RA[1:2]
rm(exp_norm)
gc()

2 x 2 sparse Matrix of class "dgCMatrix"
            BRI-399_AAACGAACAGTCTGGC BRI-399_AAAGGATGTCTCAAGT
MIR1302-2HG                        .                        .
FAM138A                            .                        .

[1] "MIR1302-2HG" "FAM138A"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2438693,130.3,4355070,232.6,4301247,229.8
Vcells,4668866,35.7,1269852816,9688.3,1321523280,10082.5


In [68]:
# see how many genes are initially different
length(gene_list_sumstats)
length(gene_list_RA)
insumstats_notsc <- setdiff(gene_list_sumstats, gene_list_RA)
length(insumstats_notsc)

[1] 18065

[1] 33538

[1] 819

### Get Synonyms of genes from NCBI

In [9]:
gene_info <- fread("/Users/hopekirby/Downloads/Homo_sapiens.gene_info")
strsplit(gene_info$Synonyms[1], "|", fixed=TRUE)
gene_split <- strsplit(gene_info$Synonyms, "|", fixed=TRUE)
full_help_list <- unlist(gene_split)
full_help_list[1:10]

[[1]]
[1] "A1B"      "ABG"      "GAB"      "HYST2477"

[1] "A1B"      "ABG"      "GAB"      "HYST2477" "A2MD"     "CPAMD5"  
 [7] "FWP007"   "S863-7"   "A2MP"     "AAC1"

In [10]:
# make a new dataframe
gene_info <- gene_info[,c("Symbol", "Synonyms")]

# add a slot as a list for each
gene_info$Syn_list <- gene_split
gene_info[1:2,]
func.text <- function(arg1, arg2) {return(rep(arg1, length(arg2)))}
gene_info[, c("Symb_list") := mapply(func.text, c(Symbol),  c(Syn_list))]
gene_info[1:2,]        

# now make a dictionary of synynonym to original
syn_dict <- do.call(c, gene_info$Symb_list)
names(syn_dict) <- do.call(c, gene_info$Syn_list)
length(syn_dict)
syn_dict[1:3]

Symbol,Synonyms,Syn_list
<chr>,<chr>,<list>
A1BG,A1B|ABG|GAB|HYST2477,"A1B , ABG , GAB , HYST2477"
A2M,A2MD|CPAMD5|FWP007|S863-7,"A2MD , CPAMD5, FWP007, S863-7"


Symbol,Synonyms,Syn_list,Symb_list
<chr>,<chr>,<list>,<list>
A1BG,A1B|ABG|GAB|HYST2477,"A1B , ABG , GAB , HYST2477","A1BG, A1BG, A1BG, A1BG"
A2M,A2MD|CPAMD5|FWP007|S863-7,"A2MD , CPAMD5, FWP007, S863-7","A2M, A2M, A2M, A2M"


[1] 195943

A1B    ABG    GAB 
"A1BG" "A1BG" "A1BG"

In [69]:
# NOW GET FIXED ONLY IF SYNONYM IN SC RNA
fixed_gene_list_sumstats <- c()
for (gene in gene_list_sumstats) {
    if (gene %in% names(syn_dict) & syn_dict[gene] %in% gene_list_RA) {
        gene <- syn_dict[gene]
    }
    fixed_gene_list_sumstats <- c(fixed_gene_list_sumstats, gene)
}
length(fixed_gene_list_sumstats)

[1] 18065

In [70]:
length(insumstats_notsc)
inncbi_sumstats_notsc <- setdiff(fixed_gene_list_sumstats, gene_list_RA)
length(inncbi_sumstats_notsc)

[1] 819

[1] 349

### Repeat but for hand identified ones (using gene cards)

In [71]:
hand_fix = c("TNFRSF6B"="RTEL1-TNFRSF6B", "CPTP"="GLTPD1", "FDX1L" = "FDX2", "FAM109A" = "PHETA1", "METTL21B" = "EEF1AKMT3", "C1orf106" = "INAVA", 
         "C21orf33" = "GATD3A", "C20orf195" = "FNDC11", "ATP5J2-PTCD1" = "ATP5MF-PTCD1", "C16orf93" = "CCDC189", 
         "ALS2CR11" = "C2CD6", 'TMEM194B' = "NEMP2", 'MRE11A' = 'MRE11', 'KIAA1107' = 'BTBD8',  
         'ATP5L' = 'ATP5MG', 'GS1-259H13.2' = 'TMEM225B', 'SDCCAG3' = 'ENTR1', 'FAM105A' = 'OTULINL', 
          'SGK223' = 'PRAG1', 'ANKRD32' = 'SLF1', 'SMEK2' = 'PPP4R3B', 'C9orf156' = 'TRMO', 
          'FAM195B' = 'MCRIP1', 'BZRAP1' = 'TSPOAP1', 'C9orf156' = 'TRMO', 'FAM195B' = 'MCRIP1', "BZRAP1"="TSPOAP1", 
             "FAM65B" = "RIPOR2", "TCEB1"="ELOC", "SEP15"="SELENOF", "C9orf171"="CFAP77", "KIAA2018"="USF3", "FAM175B"="ABRAXAS2", 
             "CASTOR3P"="GATS", "LYRM5"="ETFRF1", "CPSF3L"="INTS11", "C1orf27"="ODR4", "C1orf168"="FYB2", "PCNX"="PCNX1", 
            "KIAA0226"="RUBCN",  "TMEM133"="ARHGAP42", "C1orf95"="STUM", "C10orf54"="VSIR", "ICT1"="MRPL58", "C6orf165"="CFAP206", 
             "C9orf172"="AJM1", "C3orf17"="NEPRO", "GRAMD3"="GRAMD2B", "TMEM194A"="NEMP1", "CCDC108"="CFAP65", "FAM35A"="SHLD2", "C11orf30"="EMSY", 
             "GYLTL1B"="LARGE2", "C10orf107"="CABCOCO1", "C14orf80"="TEDC1", "SEPW1"="SELENOW", "ATP5L2"="ATP5MGL", "SLC35E2"="SLC35E2A", "C6orf183"="CCDC162P", 
             "C1orf228"="ARMH1", "LOC729800"="DMBT1L1", "IKBKAP"="ELP1", "C2orf61"="STPG4", "SRPR"="SRPRA", "SMIM11" = "SMIM11A", "C14orf105"="CCDC198", "TMEM57"="MACO1", 
     "GLTSCR2"="NOP53", "CCBL2"="KYAT3", "ATP5H"="ATP5PD", "C19orf43"="TRIR", "KIAA1161"="MYORG", "FAM63B"="MINDY2", "CCDC79"="TERB1", "CCDC176"="BBOF1", "FAM21A"="WASHC2A", 
    "SHFM1"="SEM1", "C7orf49"="CYREN", "C17orf85"="NCBP3", "GATSL3"="CASTOR1", "TCEB2"="ELOB", "CCDC109B"="MCUB", "C15orf27"="TMEM266", "C9orf91"="TMEM268", 
             "KIAA1683"="IQCN", "C1orf234"="TEX46", 
             "ATP5J2" = "ATP5MF", "N6AMT2"="EEF1AKMT1", "KIAA0368"="ECPAS", "DCDC5"="DCDC1", "SELV"="SELENOV", 
             "CCDC101"="SGF29", "C1orf111"="SPATA46", "C1orf234"="TEX46", "DYX1C1"="DNAAF4", 
             "ATHL1"="PGGHG", "TEX40"="CATSPERZ" ,
             "MEGT1"="LY6G6D", "SCAND3"="ZBED9", "DPCR1"="MUCL3", "C6orf100"="LINC01556", 
             "C6orf25"="MPIG6B", "SBP1"="VTA1", "LRRC16A"="CARMIL1", "FAM105B"="OTULIN", "C3orf83"="MKRN2OS", 
             "C20orf201"="LKAAEAR1", "C20ORF135"="ABHD16B", "KIAA1432"="RIC1", "ACPL2"="PXYLP1", "FAIM3"="FCMR", 
             "C5orf54"="ZBED8", "LINC00593"="DRAIC", "C6orf1"="SMIM29", "XRCC6BP1"="ATP23", 
             "AP000295.9"="IFNAR2", 
             "WIBG" ="PYM1", "FBXO18"="FBH1", "CLLU1" = "LINC02397", "C19orf80"="ANGPTL8", 
            "C2orf27A"="C2orf27B", "CIBAR1"="FAM92A", "ATP5MJ"="ATP5MPL", "RAMAC"="RAMMET", 
    "PELATON"="SMIM25", "GATD3"="GATD3A", "C21orf33"="GATD3A", "LOC200726"="FAM237A", 
            "AZIN2"="ADC", "SDHAF3"="ACN9", "LMNTD2"="C11orf35", 
             "DDIAS"="C11orf82", "UQCC3"="C11orf83", "TMEM263"="C12orf23", 
             "ATG101"="C12orf44", "TIGAR"="C12orf5", "CFAP54"="C12orf55", 
             "CCDC184"="C12orf68")


In [72]:
# NOW GET FIXED ONLY IF SYNONYM IN SC RNA
fixed_gene_list_sumstats2 <- c()
for (gene in fixed_gene_list_sumstats) {
    if (gene %in% names(hand_fix) & hand_fix[gene] %in% gene_list_RA) {
        # if (gene %in% inncbihand_sumstats_notsc) {print(gene)
        #                                           print(hand_fix[gene])}
        gene <- hand_fix[gene]
    }
    fixed_gene_list_sumstats2 <- c(fixed_gene_list_sumstats2, gene)
}
length(fixed_gene_list_sumstats2)

[1] 18065

In [73]:
length(insumstats_notsc)
length(inncbi_sumstats_notsc)
inncbihand_sumstats_notsc <- setdiff(fixed_gene_list_sumstats2[1:1020], gene_list_RA)
length(inncbihand_sumstats_notsc)

[1] 819

[1] 349

[1] 12

In [ ]:
c("FLJ46235") %in% gene_list_RA

In [54]:
#before <- inncbihand_sumstats_notsc
before <- c(before, inncbihand_sumstats_notsc)

In [74]:
inncbihand_sumstats_notsc
setdiff(inncbihand_sumstats_notsc, before)

[1] "LOC100996337" "LOC730183"    "LOC101928947" "LOC101927260" "LOC101929578"
 [6] "LOC101060024" "LOC390877"    "LOC101929256" "LOC101927160" "LOC101928021"
[11] "LOC441239"    "LOC728715"

[1] "LOC728715"

### See how much lack of genes can be explained by non-protein genes

In [ ]:
# length(inhandfixedM10_notours)
# length(grep("-AS",inhandfixedM10_notours))
# length(grep("[0-9].[0-9]",inhandfixedM10_notours))

# # remove antisense
# still_not <- setdiff(inhandfixedM10_notours, 
#                      inhandfixedM10_notours[grep("-AS",inhandfixedM10_notours)])
# # remove X.X
# still_not <- setdiff(still_not, 
#                      inhandfixedM10_notours[grep("[0-9].[0-9]",inhandfixedM10_notours)])
# length(still_not)
#still_not

length(inhandfixedM53_notours)
length(grep("-AS",inhandfixedM53_notours))
length(grep("[0-9].[0-9]",inhandfixedM53_notours))

# remove antisense

still_not <- setdiff(inhandfixedM53_notours, 
                     inhandfixedM53_notours[grep("-AS",inhandfixedM53_notours)])
# remove X.X
still_not <- setdiff(still_not, 
                     inhandfixedM53_notours[grep("[0-9].[0-9]",inhandfixedM53_notours)])
length(still_not)
#still_not


In [ ]:
# see which genes are in hg37 but not hg38
#intersect(still_not, setdiff(hg37_list, hg38_list))
#length(setdiff(gene_list, hg37_list))
length(intersect(genes_out_53$SYMBOL, setdiff(gene_list, hg37_list)))
print("Which genes are in hg37 but not found in gene list, intersected with those found in genelist but not hg37")
length(intersect(still_not, setdiff(gene_list, hg37_list)))
length(setdiff(genes_out_53$SYMBOL, hg37_list))
length(setdiff(genes_out_53$SYMBOL, hg38_list))

In [ ]:
# A 0 indicates that the genes found in MAGMA but not the gene list 

In [ ]:
genes_out_53[1:2,]

In [ ]:
top_not <- setdiff(genes_out_53$SYMBOL[1:1000], gene_list)
length(top_not)

In [ ]:
length(gene_list)

In [ ]:
length(setdiff(gene_list, hg37_list))
length(setdiff(gene_list, hg38_list))
length(intersect(setdiff(gene_list, hg37_list), 
                 setdiff(gene_list, hg38_list)))
in_gene_list_not_magma <- intersect(setdiff(gene_list, hg37_list), 
                 setdiff(gene_list, hg38_list))

In [ ]:
# see how much 
length(in_gene_list_not_magma)
length(grep("-AS",in_gene_list_not_magma))
length(grep("[0-9].[0-9]",in_gene_list_not_magma))
length(grep("-IT1",in_gene_list_not_magma))

# remove antisense

still_not <- setdiff(in_gene_list_not_magma, 
                     in_gene_list_not_magma[grep("-AS",in_gene_list_not_magma)])
# remove X.X
still_not <- setdiff(still_not, 
                     in_gene_list_not_magma[grep("[0-9].[0-9]",in_gene_list_not_magma)])
still_not <- setdiff(still_not, 
                     in_gene_list_not_magma[grep("-IT1",in_gene_list_not_magma)])
length(still_not)
still_not[301:400]


In [ ]:
test <- c("TCRGC2", "TARP", "TCRG")
test %in% genes_out_53$SYMBOL
test %in% gene_list
test %in% hg37_list
# test %in% hg38_list

## Remove duplicates by keeping one with lowest p-value

In [75]:
genes_out$fixed <- fixed_gene_list_sumstats2

In [76]:
genes_out[genes_out$fixed %in% genes_out$fixed[duplicated(genes_out$fixed)][1:5],]


GENE,CHR,START,STOP,NSNPS,NPARAM,N,ZSTAT,P,SYMBOL,fixed
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>
2214,1,161501549,161530413,104,27,97173,5.5192,1.7032e-08,FCGR3A,FCGR3B
9103,1,161541129,161581010,160,28,97173,4.0993,2.0720e-05,FCGR2C,FCGR2B
8663,16,28689879,28757051,39,11,97173,3.9413,4.0515e-05,EIF3C,EIF3C
728689,16,28380900,28425162,24,4,97173,3.3034,4.7767e-04,EIF3CL,EIF3C
284697,1,92535862,92623397,254,23,97173,3.1598,7.8940e-04,BTBD8,BTBD8
2213,1,161622905,161658444,124,30,97173,3.1319,8.6831e-04,FCGR2B,FCGR2B
100861540,7,99272302,99342819,184,19,97173,3.0393,1.1857e-03,CYP3A7-CYP3A51P,CYP3A7-CYP3A51P
1551,7,99292660,99342853,137,13,97173,3.0207,1.2610e-03,CYP3A7,CYP3A7-CYP3A51P
2215,1,161582986,161611753,259,18,97173,2.9532,1.5726e-03,FCGR3B,FCGR3B


In [77]:
length(genes_out$fixed[duplicated(genes_out$fixed)])
dim(genes_out)

[1] 690

[1] 18065    11

In [78]:
# for any duplications, just keep the one with the lowest p-value
remove_duplicate_genes <- function(df) {
    repeats <- df$fixed[duplicated(df$fixed)]
    print(length(repeats))
non_dup <- df[!(df$fixed %in% repeats),]
    print(dim(non_dup))
seq_keep <- c()
for (geneid in repeats) {
    filtered <- df[df$fixed == geneid,]
    filtered <- filtered[order(-filtered$P),]
    seq_keep <- c(seq_keep, filtered[1,c("unique_id")])
    }
fixeddf <- df[df$unique_id %in% seq_keep,]
    fixeddf <- rbind(non_dup, fixeddf)
    if (length(fixeddf$fixed[duplicated(fixeddf$fixed)]) > 0) {print("WARNING")}
    fixeddf
    }

length(genes_out$fixed[duplicated(genes_out$fixed)])
# unique id
genes_out$unique_id <- seq(1,nrow(genes_out))

dim(genes_out)
genes_out_nondup <- remove_duplicate_genes(genes_out)
dim(genes_out_nondup)

[1] 690

[1] 18065    12

[1] 690
[1] 16706    12


[1] 17375    12

In [79]:
length(genes_out_nondup$fixed[duplicated(genes_out_nondup$fixed)])

[1] 0

## Save Files

In [80]:
# Now make a file of the Gene, and P values of the traits
gene_pval <- genes_out_nondup[,c("fixed", "P")]
colnames(gene_pval) <- c("GENE", "RA")

gene_full <- genes_out_nondup[,c("fixed", "CHR", "START", 
                          "STOP", "NSNPS", "NPARAM", 
                          "N", "ZSTAT", "P")]
colnames(gene_full) <- c("GENE", "CHR", "START", 
                          "STOP", "NSNPS", "NPARAM", 
                          "N", "ZSTAT", "P")

In [81]:
# remove any ones not in the dataset
final_gene_pval <- gene_pval[gene_pval$GENE %in% gene_list_RA,]
dim(gene_pval)
dim(final_gene_pval)

[1] 17375     2

[1] 17032     2

In [82]:
# save files
# save the files as RA_ENSEMBL_Pval_04-18-23.tsv & RA_ENSEMBL_04-18-23_Z.tsv
dir <- "/Users/hopekirby/Desktop/SCRNA-GWAS-Benchmarking/output/MAGMA/RA/"
write.table(gene_full, 
            paste0(dir, "RA_10kb_full_nodups_07-09-23.tsv"), sep='\t', quote=FALSE, row.names=FALSE)
write.table(gene_pval, 
            paste0(dir, "RA_10kb_Pval_full_nodups_07-09-23.tsv"), sep='\t', quote=FALSE, row.names=FALSE)

write.table(final_gene_pval, 
            paste0(dir, "RA_10kb_Pval_onlyinrna_nodups_07-09-23.tsv"), sep='\t', quote=FALSE, row.names=FALSE)

